# Hydrograph and Seatek Sensor Analysis

This notebook demonstrates the analysis workflow for river bed monitoring data.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_processing import DataProcessor
from src.visualization import Visualizer


## Load and Process Data

In [ ]:
def load_and_process_data(file_path, river_mile):
    processor = DataProcessor(file_path)
    processor.load_data()
    return processor.process_river_mile(river_mile)

rm_54_data = load_and_process_data("data/sensor_readings.xlsx", 54.0)

## Basic Analysis

In [ ]:
def calculate_basic_stats(data):
    stats = {
        'mean_depth': data['Sensor_1'].mean(),
        'std_depth': data['Sensor_1'].std(),
        'max_change': np.diff(data['Sensor_1']).max()
    }
    return stats

stats = calculate_basic_stats(rm_54_data)
print(f"Mean depth: {stats['mean_depth']:.2f} mm")
print(f"Depth standard deviation: {stats['std_depth']:.2f} mm")
print(f"Maximum change rate: {stats['max_change']:.2f} mm/5min")

## Visualizations

In [ ]:
def create_time_series_plot(data, river_mile, year, sensor):
    viz = Visualizer()
    fig, stats = viz.create_visualization(data, river_mile, year, sensor)
    threshold = stats['mean'] + 2 * stats['std']
    significant_events = data[data[sensor] > threshold]
    plt.scatter(significant_events.index, significant_events[sensor], color='red', label='Significant Events')
    plt.legend()
    plt.savefig(f'output/{sensor}_time_series_plot.png', dpi=300)
    return fig, stats

fig, stats = create_time_series_plot(rm_54_data, 54.0, 2023, 'Sensor_1')

## Environmental Impact Analysis

In [ ]:
def analyze_bed_stability(data, window='24H'):
    rolling_std = data['Sensor_1'].rolling(window).std()
    stability_categories = pd.cut(rolling_std, bins=[0, 5, 20, np.inf], labels=['Stable', 'Moderate', 'Active'])
    return stability_categories

stability = analyze_bed_stability(rm_54_data)
print("\nBed Stability Analysis:")
print(stability.value_counts())

## Correlation Analysis

In [ ]:
def plot_sensor_correlation(data, sensor1, sensor2):
    correlation = data[sensor1].corr(data[sensor2])
    print(f"\nSensor Correlation: {correlation:.2f}")
    plt.figure(figsize=(10, 6))
    plt.scatter(data[sensor1], data[sensor2], alpha=0.5)
    plt.xlabel(f'{sensor1} (mm)')
    plt.ylabel(f'{sensor2} (mm)')
    plt.title('Sensor Correlation Analysis')
    plt.savefig('output/correlation_analysis.png', dpi=300)

plot_sensor_correlation(rm_54_data, 'Sensor_1', 'Sensor_2')

## Export Results

In [ ]:
def save_processed_data(data, file_path):
    try:
        data.to_excel(file_path)
    except Exception as e:
        print(f"Error saving data: {e}")

save_processed_data(rm_54_data, 'output/processed_data.xlsx')

This notebook provides a template for analyzing river bed monitoring data, including:

    - Data loading and preprocessing
    - Basic statistical analysis
    - Visualization of time series data
    - Environmental stability analysis
    - Sensor correlation analysis